# Imports and utilities

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from shared.snowflake.client import SnowflakeClient

client = SnowflakeClient()

In [3]:
session = client.get_snowpark_session()

In [4]:
import logging

# 1) Configure YOUR logs
logging.basicConfig(
    level=logging.INFO,  # <-- important: don't set root to DEBUG
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,
)

# 2) Silence noisy libraries
for noisy in [
    "snowflake",
    "snowflake.connector",
    "snowflake.snowpark",
    "urllib3",
]:
    logging.getLogger(noisy).setLevel(logging.WARNING)

# (Optional) If something still leaks through, also:
logging.getLogger("snowflake.connector.cursor").setLevel(logging.WARNING)
logging.getLogger("snowflake.connector.connection").setLevel(logging.WARNING)
logging.getLogger("snowflake.snowpark._internal.server_connection").setLevel(
    logging.WARNING
)

In [5]:
# Transform service dependencies
from app.services.transform_service import TransformService
from app.models.transform import (
    TransformConstraints,
    TransformationType,
    TransformRequest,
    Recipe,
)

# Snowflake example usage

In [37]:
client.is_connected()

{'ok': True, 'version': '9.39.2'}

In [38]:
query = """
    SELECT 
        ID,
        NAME,
        INGREDIENTS,
        STEPS,
        MINUTES,
    FROM NUTRIRAG_PROJECT.RAW.RAW_RECIPES_10K
    LIMIT 1
"""

result_df = client.execute(query, fetch="one")

# Transform service example usage

In [6]:
transform_service = TransformService(session)

In [ ]:
import pandas as pd

transform_service.pca_data = pd.read_csv("ingredients_tags.csv").iloc[:, 1:]

### Input definition

In [38]:
request_fmt = "class"

ingredients = [
    "celery",
    "onion",
    "cooked brown rice",
    "vegetarian chicken substitute",
    "mushroom soup",
    "mushrooms",
    "soymilk",
    "soy mayonnaise",
    "salt",
    "paprika",
    "sliced almonds",
]

steps = [
    "saute celery and onion in a very small amount of oil or water",
    "put into mixing bowl and add remaining ingredients , mixing well",
    "sprinkle top with paprika and sliced almonds",
    "put in oiled casserole dish and bake at 350 degrees f about 45 minutes",
]


ingredients_to_remove = None  # ["mushrooms"]

if request_fmt == "snow":
    recipe = {
        "id": 241686,
        "name": "heavenly  chicken   frichik  casserole",
        "serving_size": 182,
        "servings": 8,
        "ingredients": ingredients,
        "quantity_ingredients": ["1"] * len(ingredients),
        "minutes": 70.0,
        "steps": steps,
        "health_score": 57,
    }

    constraints = {
        "transformation": 2,
        "no_lactose": False,
        "no_gluten": False,
        "no_nuts": False,
        "vegetarian": False,
        "vegan": False,
        "increase_protein": False,
        "decrease_sugar": False,
        "decrease_protein": False,
        "decrease_carbs": False,
        "decrease_calories": False,
        "decrease_sodium": False,
    }

    request = {
        "recipe": recipe,
        "ingredients_to_remove": ingredients_to_remove,
        "constraints": constraints,
    }

else:
    recipe_obj = Recipe(
        id=241686,
        name="heavenly  chicken   frichik  casserole",
        serving_size=182,
        servings=8,
        ingredients=ingredients,
        quantity_ingredients=["10"] * len(ingredients),
        minutes=70.0,
        steps=steps,
        health_score=57,
    )

    constraints_obj = TransformConstraints(
        transformation=TransformationType.DELETE,
        no_lactose=False,
        no_gluten=False,
        no_nuts=False,
        vegetarian=True,
        vegan=False,
        increase_protein=False,
        decrease_sugar=False,
        decrease_protein=False,
        decrease_carbs=False,
        decrease_calories=False,
        decrease_sodium=False,
    )

    request = TransformRequest(
        recipe=recipe_obj,
        ingredients_to_remove=ingredients_to_remove,
        constraints=constraints_obj,
    )

### Run Transform service

In [39]:
result = transform_service.transform(
    recipe=request.recipe,
    ingredients_to_remove=request.ingredients_to_remove,
    constraints=request.constraints,
)

Step 1a: Identification by algorithm...


Ingredients identified: ['celery', 'onion', 'cooked brown rice']


2026-01-11 19:40:40,552 - root - INFO - Success: Step 1 finished (Ingredients to remove has been found).
2026-01-11 19:40:43,002 - root - INFO - Success: Step 2 finished for Deletion (Removed successfully unwanted ingredients and computed new health score).


LLM: 5 adapted steps, 0 notes


2026-01-11 19:40:46,402 - root - INFO - Success: Step 3 finished for Deletion (LLM's adapted new_recipe steps successfully).
2026-01-11 19:40:48,954 - root - INFO - Success: Step 4 finished (Original recipe health score computing finished).
2026-01-11 19:40:48,957 - root - INFO - Success: Step 5 finished (TransformerResponse successfully built, returning it...).


In [40]:
vars(result)

{'recipe': Recipe(id=241686, name='heavenly  chicken   frichik  casserole', serving_size=182.0, servings=8.0, health_score=42.21431691648693, ingredients=['vegetarian chicken substitute', 'mushroom soup', 'mushrooms', 'soymilk', 'soy mayonnaise', 'salt', 'paprika', 'sliced almonds'], quantity_ingredients=['10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10'], minutes=70.0, steps=["Put all the remaining ingredients, excluding 'celery', 'onion', and 'cooked brown rice', into a mixing bowl.", 'Mix all the ingredients in the bowl well, ensuring that the vegetarian chicken substitute is evenly distributed.', 'Transfer the mixed ingredients into an oiled casserole dish.', 'Sprinkle the top with paprika and sliced almonds for added flavor and a crunchy texture.', 'Bake in the oven at 350 degrees Fahrenheit for approximately 45 minutes.']),
 'original_name': 'heavenly  chicken   frichik  casserole',
 'transformed_name': 'heavenly  chicken   frichik  casserole',
 'substitutions': No

In [41]:
ingredients

['celery',
 'onion',
 'cooked brown rice',
 'vegetarian chicken substitute',
 'mushroom soup',
 'mushrooms',
 'soymilk',
 'soy mayonnaise',
 'salt',
 'paprika',
 'sliced almonds']

In [42]:
result.recipe.ingredients

['vegetarian chicken substitute',
 'mushroom soup',
 'mushrooms',
 'soymilk',
 'soy mayonnaise',
 'salt',
 'paprika',
 'sliced almonds']

In [25]:
matchs = transform_service.get_ingredient_matched(ingredients)

2026-01-11 19:30:52,090 - root - INFO - Matching: Looking for ingredients: ['celery', 'onion', 'cooked brown rice', 'vegetarian chicken substitute', 'mushroom soup', 'mushrooms', 'soymilk', 'soy mayonnaise', 'salt', 'paprika', 'sliced almonds'].
2026-01-11 19:30:53,635 - root - INFO - result: [{'DESCRIP': 'onions raw', 'PROTEIN_G': Decimal('1.10'), 'SATURATED_FATS_G': Decimal('0.042000000000000000'), 'FAT_G': Decimal('0.10'), 'CARB_G': Decimal('9.34'), 'SODIUM_MG': Decimal('4.00'), 'FIBER_G': Decimal('1.70'), 'SUGAR_G': Decimal('4.24'), 'ENERGY_KCAL': Decimal('40.000000000000000'), 'MATCHED_INGREDIENT': 'green onions'}, {'DESCRIP': 'silk vanilla soymilk', 'PROTEIN_G': Decimal('2.47'), 'SATURATED_FATS_G': Decimal('0.206000000000000020'), 'FAT_G': Decimal('1.44'), 'CARB_G': Decimal('4.12'), 'SODIUM_MG': Decimal('39.00'), 'FIBER_G': Decimal('0.40'), 'SUGAR_G': Decimal('2.88'), 'ENERGY_KCAL': Decimal('41.000000000000000'), 'MATCHED_INGREDIENT': 'vanilla'}, {'DESCRIP': 'paprika', 'PROTEIN_G

In [28]:
ingr_match = [d["name"] for d in matchs]

In [20]:
pca_data = transform_service.pca_data

In [30]:
ingredients

['celery',
 'onion',
 'cooked brown rice',
 'vegetarian chicken substitute',
 'mushroom soup',
 'mushrooms',
 'soymilk',
 'soy mayonnaise',
 'salt',
 'paprika',
 'sliced almonds']

In [29]:
pca_data[pca_data["Descrip"].isin(ingr_match)]

,NDB_NO,Descrip,ENERGY_KCAL,PROTEIN_G,SATURATED_FATS_G,FAT_G,CARB_G,SODIUM_MG,SUGAR_G,PCA_macro_1,...,PCA_macro_3,PCA_micro_1,PCA_micro_2,CLUSTER_MACRO,CLUSTER_MICRO,is_lactose,is_gluten,contains_nuts,is_vegetarian,is_vegetable
256,2007,celery seed,392.000000000000000,18.07,2.1900000000000000,25.27,41.35,160.00,0.67,1.41010092566738700000,...,2.19851534553872500000,8,-6.26806186878592800000,2,0,0,0,0,1,0
277,2028,paprika,282.000000000000000,14.14,2.1400000000000000,12.89,53.99,68.00,10.34,1.71572645627592360000,...,5.29284604585206800000,7,-1.34113880491569180000,2,0,0,0,0,1,0
295,2047,salt table,0E-15,0.00,0E-16,0.00,0.00,38758.00,0.00,-1.76016727026789390000,...,-0.96079406334584550000,0,-3.35611625956113800000,0,0,0,0,0,1,0
1290,6147,soup bf mushroom cnd cond,61.000000000000000,4.60,1.2000000000000000,2.40,5.20,709.00,0.00,-1.32653823614768540000,...,-0.63714596324435400000,-1,0.38614696171674300000,0,0,0,0,0,1,0
3600,11993,mushrooms maitake raw,31.000000000000000,1.94,0.0300000000000000,0.19,6.97,1.00,2.07,-1.44033126384401780000,...,-0.43319815185097016000,0,1.97404556460439770000,0,0,0,0,0,1,0
3634,12061,almonds,579.000000000000000,21.15,3.8020000000000000,49.93,21.55,1.00,4.35,2.84233598499339870000,...,2.34021675645963170000,4,-1.98484157028969730000,1,0,0,0,1,1,0
4913,16223,soymilk (all flavors) enhanced,45.000000000000000,2.94,0.2060000000000000,1.99,3.45,50.00,2.53,-1.42807039577812000000,...,-0.75221842901353960000,-1,0.40130876504685364000,0,0,1,0,0,1,0
8589,42178,mayonnaise made with tofu,322.000000000000000,5.95,2.9210000000000003,31.79,3.06,773.00,0.85,0.67903679659460700000,...,-0.39446332522840394000,-1,0.01006963249191906600,0,0,0,0,0,1,0
8663,43128,chicken meatless,224.000000000000000,23.64,1.5340000000000000,12.73,3.64,709.00,0.00,-0.34818277951015086000,...,1.06977761285340130000,0,0.33431161073940235000,0,0,0,0,0,0,0
8802,A013,rice raw brown,353.720000000000000,9.16,0.3460000000000000,1.24,74.80,3.64,0.69,0.52426114572670020000,...,0.78274220340259690000,0,-0.31738453835399255000,2,0,0,0,0,1,0


In [53]:
vars(request)

{'recipe': Recipe(id=94947, name='crab filled crescent snacks', serving_size=1.0, servings=4.0, health_score=57.0, ingredients=['crabmeat', 'cream cheese', 'green onions', 'garlic salt', 'refrigerated crescent dinner rolls', 'egg yolk', 'water', 'sesame seeds', 'sweet and sour sauce'], quantity_ingredients=['1', '1', '1', '1', '1', '1', '1', '1', '1'], minutes=70.0, steps=['heat over to 375 degrees', 'spray large cookie sheet with non-stick cooking spray', 'in small bowl , combine crabmeat , cream cheese , onions and garlic salt and mix well', 'unroll both cans of dough', 'separate into 16 triangles', 'cut each triangle in half lengthwise to make 32 triangles', 'place 1 teaspoon crab mixture on center of each triangle about 1 inch from short side of triangle', 'fold short ends of each triangle over filling', 'pinch sides to seal', 'roll up', 'place on sprayed cookie sheet', 'in small bowl , combine egg yolk and water and mix well', 'brush egg mixture over snacks', 'sprinkle with sesame

In [54]:
result.success

True

# Always close connexion before closing notebook

In [68]:
client.close()